# Data collection of soccer players

## Libraries

In [4]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split as tts
from scipy.stats import norm, skew, kurtosis, anderson, kstest
import seaborn as sns

## Open csv files

In [5]:
age_df = pd.read_csv("AGE.csv")
date_of_birth =  pd.read_csv('players.csv', usecols= ['player_id', 'birth_date'])
mv_df =  pd.read_csv('market_value_df.csv')
player_data =  pd.read_csv('player table189.csv')
stc_data = pd.read_csv('statistics table189.csv')

### clean age_df data

In [6]:
age_df.loc[age_df['Age'].str.contains('†'), 'Age'] = age_df['Age'].str.replace('†', '')

### clean player_df data

In [7]:
player_df = player_data.drop(player_data.columns[0], axis=1)
player_df = player_df.drop('birth_date', axis=1)

height_median = player_df['height'].median()
# Replace NaN values with the median of the 'height' column
player_df['height'] = player_df['height'].fillna(height_median)

player_df['main_position'].replace(['Attack Left Winger','Attack Centre Forward', 'Attack Right Winger', 'Attack Second Striker'], 'striker', inplace=True)
player_df['main_position'].replace(['Defender Centre Back', 'Defender Right Back', 'Defender Left Back'], 'defender', inplace=True)
player_df['main_position'].replace(['midfield Central Midfield', 'midfield Attacking Midfield', 'midfield Defensive Midfield', 'midfield', 'midfield Left Midfield', 'midfield Right Midfield'], 'midfielder', inplace=True)

player_df['goals_scored'].replace('-', np.nan, inplace=True)
player_df['goals_assisted'].replace('-', np.nan, inplace=True)
player_df['total_appearence'].replace('-', np.nan, inplace=True)
player_df['agent'] = player_df['agent'].replace('', np.nan)

player_df['goals_conceded'] = np.where(player_df['main_position'] != 'Goalkeeper', 'not defined',
                       np.where(player_df['goals_conceded'] == '-', np.nan, player_df['goals_conceded']))

player_df['clean_sheets'] = np.where(player_df['main_position'] != 'Goalkeeper', 'not defined',
                       np.where(player_df['clean_sheets'] == '-', np.nan, player_df['clean_sheets']))

In [8]:
print(age_df[age_df['Age'].str.contains('†', na=False)])
print(player_df['main_position'].unique())

Empty DataFrame
Columns: [season, Age, player_id, season_short]
Index: []
['Goalkeeper' 'defender' 'midfielder' 'striker']


### clean stc_df data

In [9]:
stc_df = stc_data.drop(stc_data.columns[0], axis=1)

stc_df['Appearances'].replace('-', np.nan, inplace=True)
stc_df['PPG'].replace('-', np.nan, inplace=True)
stc_df['PPG'].replace('0,00', 0, inplace=True)
stc_df['Goals'].replace('-', np.nan, inplace=True)
stc_df['Own goals'].replace('-', np.nan, inplace=True)
stc_df['Substitutions on'].replace('-', np.nan, inplace=True)
stc_df['Substitutions off'].replace('-', np.nan, inplace=True)
stc_df['Yellow cards'].replace('-', np.nan, inplace=True)
stc_df['Second yellow cards'].replace('-', np.nan, inplace=True)
stc_df['Red cards'].replace('-', np.nan, inplace=True)
stc_df['Assists'].replace('-', np.nan, inplace=True)
stc_df['Penalty goals'].replace('-', np.nan, inplace=True)
stc_df['Minutes per goal'].replace('-', np.nan, inplace=True)

stc_df = pd.merge(stc_df, player_df[['player_id', 'main_position']], on='player_id', suffixes=('_stc_df', '_player_df'))
stc_df['Goals conceded'] = np.where(stc_df['main_position'] != 'Goalkeeper', 'not defined',
                       np.where(stc_df['Goals conceded'] == '-', np.nan, stc_df['Goals conceded']))

stc_df['Clean sheets'] = np.where(stc_df['main_position'] != 'Goalkeeper', 'not defined',
                       np.where(stc_df['Clean sheets'] == '-', np.nan, stc_df['Clean sheets']))

ppg_median = stc_df['PPG'].median()
# Replace NaN values with the median of the 'PPG' column
stc_df['PPG'] = stc_df['PPG'].fillna(ppg_median)

In [10]:
print(stc_df.dtypes)

player_id               int64
Season                 object
Competition            object
Club                   object
Squad                   int64
Appearances            object
PPG                    object
Goals                  object
Own goals              object
Substitutions on       object
Substitutions off      object
Yellow cards           object
Second yellow cards    object
Red cards              object
Goals conceded         object
Clean sheets           object
Minutes played         object
Assists                object
Penalty goals          object
Minutes per goal       object
main_position          object
dtype: object


In [59]:
stc_df.shape

(50187, 21)

In [17]:
np.shape(mv_df)

(12594, 3)

In [39]:
np.shape(stc_df)

(50187, 21)

In [60]:
Goalkeeper_df = stc_df[stc_df['main_position'] == 'Goalkeeper'] 
striker_df = stc_df[stc_df['main_position'] == 'striker'] 
defender_df = stc_df[stc_df['main_position'] == 'defender'] 
midfielder_df = stc_df[stc_df['main_position'] == 'midfielder'] 

In [61]:
np.unique(Goalkeeper_df['main_position'])

array(['Goalkeeper'], dtype=object)

In [63]:
Goalkeeper_df.shape

(5831, 21)

In [64]:
Goalkeeper_df.drop(['Goals', 'Own goals', 'Substitutions on', 'Substitutions off',
                   'Yellow cards', 'Second yellow cards', 'Red cards', 'Minutes played',
                   ], inplace=True, axis=1)

/tmp/ipykernel_42960/4223849242.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Goalkeeper_df.drop(['Goals', 'Own goals', 'Substitutions on', 'Substitutions off',


In [65]:
Goalkeeper_df_m = pd.merge(Goalkeeper_df, mv_df, on='player_id', how='left')

In [67]:
Goalkeeper_df_m.shape

(10810, 15)

In [55]:
Goalkeeper_df = Goalkeeper_df.dropna(subset=['market_value'])
Goalkeeper_df

,player_id,Season,Competition,Club,Squad,Appearances,PPG,Goals conceded,Clean sheets,Assists,Penalty goals,Minutes per goal,main_position,season,market_value
28,404839,21/22,Champions League,Real Madrid,13,NaN,1.5,NaN,NaN,NaN,NaN,NaN,Goalkeeper,2019.0,2700000.0
29,404839,21/22,Champions League,Real Madrid,13,NaN,1.5,NaN,NaN,NaN,NaN,NaN,Goalkeeper,2020.0,3000000.0
30,404839,21/22,Champions League,Real Madrid,13,NaN,1.5,NaN,NaN,NaN,NaN,NaN,Goalkeeper,2021.0,2500000.0
31,404839,21/22,LaLiga,Real Madrid,34,2,0.50,2,NaN,NaN,NaN,NaN,Goalkeeper,2019.0,2700000.0
32,404839,21/22,LaLiga,Real Madrid,34,2,0.50,2,NaN,NaN,NaN,NaN,Goalkeeper,2020.0,3000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10784,285033,16/17,EFL Trophy,Cambridge United,4,4,1.50,4,1,NaN,NaN,NaN,Goalkeeper,2021.0,300000.0
10785,285033,16/17,EFL Cup,Cambridge United,1,1,0,2,NaN,NaN,NaN,NaN,Goalkeeper,2021.0,300000.0
10786,285033,15/16,League Two,Cambridge United,21,21,1.52,17,9,NaN,NaN,NaN,Goalkeeper,2021.0,300000.0
10787,285033,15/16,National League,Braintree Town,24,24,1.67,20,10,NaN,NaN,NaN,Goalkeeper,2021.0,300000.0
